In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import requests, datetime

In [3]:
website_url='https://boardgamegeek.com/browse/boardgame/page/'

In [4]:
response = requests.get(website_url)
response.status_code

200

In [5]:
print(type(response))
response.content

<class 'requests.models.Response'>


b'\n<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tvar width = document.documentElement.clientWidth || window.innerWidth;\n\t\t\t\tif (width < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\t// android debugging\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https:

In [6]:
gameRankings = 'https://boardgamegeek.com/browse/boardgame/page/'

In [7]:
rowDomId = 'row_'
rankObjectClass = 'collection_rank'
idObjectClass = 'collection_objectname'
ratingObjectClass = 'collection_bggrating'
price ='collection_shop'

In [8]:
def get_page(pageNo):
    """This is the webscrape input is "Browse" page number, output is a list of lists of items to collect metadata for"""
    records = []
    importDate = datetime.datetime.strftime(datetime.datetime.now().date(), '%Y-%m-%d')
    req = requests.get(gameRankings + str(pageNo))
    # make sure we get back valid 200 responses
    if req.status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')
        rows = soup.find_all(id=rowDomId)
        for row in rows:
            listify = []
            # this searches for a TD element with the class == rankObjectClass
            rankTarget = row.find('td', rankObjectClass)
            if rankTarget:
                listify.append(int(rankTarget.text.strip()))
            # this searches for a TD element with the class == idObjectClass it grabs the ID & Name
            idTarget = row.find('td', idObjectClass)
            if idTarget:
                # ID 
                listify.append((idTarget.find('a')['href']).split('/')[2])
                # Name
                listify.append(idTarget.find('a').text)
                # Description - it is the only paragraoph tag in this section - sometimes not populated
                if idTarget.find('p'):
                    listify.append(idTarget.find('p').getText().strip())
                else:
                    listify.append(None)
            # this colects the 3 td's with the bggRating, avgRating, and numVotes they all use a class of collection_bggrating
            rateTarget = row.find_all('td', ratingObjectClass)
            if len(rateTarget) == 3:
                for target in rateTarget:
                    if target.text.strip().replace('.', '',1).isdigit():
                        listify.append(target.text.strip())
                    else:
                        listify.append(None)
            # if some are missing we have no way of knowing which so we must make all null
            else:
                listify = listify + [None, None, None]
            # add the import date
            listify.append(importDate)
            # check to make sure all record elements are present
            if len(listify) == 8:
                records.append(tuple(listify))
            else:
                # check to make sure all records have the proper amount of elements
                raise Exception(f'The following record is too long/short: {listify} does not have match the rankings format: rank, game_id, title, description, bgg_rating, avg_rating, votes, import_date')
        return tuple(records)


In [9]:
if __name__ == "__main__":
    results = get_page(2)
    for i in results:
        print(i)

(101, '225694', 'Decrypto', "Decipher your opponents' code before they decipher yours. Don't get caught.", '7.504', '7.80', '12993', '2021-08-11')
(102, '36218', 'Dominion', 'Acquire the most valuable lands by building your deck with treasure and power cards.', '7.504', '7.61', '79802', '2021-08-11')
(103, '2511', 'Sherlock Holmes Consulting Detective: The Thames Murders & Other Cases', 'The game is afoot as you and your irregular friends search Victorian London for clues', '7.499', '7.72', '17921', '2021-08-11')
(104, '30549', 'Pandemic', 'Your team of experts must prevent the world from succumbing to a viral pandemic.', '7.498', '7.60', '105650', '2021-08-11')
(105, '155821', 'Inis', 'Claim the crown through merit and wisdom in this Celtic island struggle.', '7.494', '7.81', '13571', '2021-08-11')
(106, '198928', 'Pandemic: Iberia', 'Travel from Barcelona to Lisboa to cure malaria, typhus, yellow fever, and cholera.', '7.490', '7.92', '8624', '2021-08-11')
(107, '172287', 'Champions 

In [10]:
results

((101,
  '225694',
  'Decrypto',
  "Decipher your opponents' code before they decipher yours. Don't get caught.",
  '7.504',
  '7.80',
  '12993',
  '2021-08-11'),
 (102,
  '36218',
  'Dominion',
  'Acquire the most valuable lands by building your deck with treasure and power cards.',
  '7.504',
  '7.61',
  '79802',
  '2021-08-11'),
 (103,
  '2511',
  'Sherlock Holmes Consulting Detective: The Thames Murders & Other Cases',
  'The game is afoot as you and your irregular friends search Victorian London for clues',
  '7.499',
  '7.72',
  '17921',
  '2021-08-11'),
 (104,
  '30549',
  'Pandemic',
  'Your team of experts must prevent the world from succumbing to a viral pandemic.',
  '7.498',
  '7.60',
  '105650',
  '2021-08-11'),
 (105,
  '155821',
  'Inis',
  'Claim the crown through merit and wisdom in this Celtic island struggle.',
  '7.494',
  '7.81',
  '13571',
  '2021-08-11'),
 (106,
  '198928',
  'Pandemic: Iberia',
  'Travel from Barcelona to Lisboa to cure malaria, typhus, yellow f

In [11]:
ranking_list=[]

for i in range(5):
    i+=1
    ranking_list.append(get_page(i))
    

ranking_list

[((1,
   '174430',
   'Gloomhaven',
   'Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!',
   '8.533',
   '8.77',
   '45001',
   '2021-08-11'),
  (2,
   '161936',
   'Pandemic Legacy: Season 1',
   'Mutating diseases are spreading around the world - can your team save humanity?',
   '8.454',
   '8.61',
   '43380',
   '2021-08-11'),
  (3,
   '224517',
   'Brass: Birmingham',
   'Build networks, grow industries, and navigate the world of the Industrial Revolution.',
   '8.402',
   '8.67',
   '22412',
   '2021-08-11'),
  (4,
   '167791',
   'Terraforming Mars',
   'Compete with rival CEOs to make Mars habitable and build your corporate empire.',
   '8.279',
   '8.43',
   '69446',
   '2021-08-11'),
  (5,
   '291457',
   'Gloomhaven: Jaws of the Lion',
   'Vanquish monsters with strategic cardplay in a 25-scenario Gloomhaven campaign.',
   '8.256',
   '8.77',
   '12214',
   '2021-08-11'),
  (6,
   '233078',
   'Twilight Imperium: Fourth Edition',
   'Build

In [12]:
get_page(1)

((1,
  '174430',
  'Gloomhaven',
  'Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!',
  '8.533',
  '8.77',
  '45001',
  '2021-08-11'),
 (2,
  '161936',
  'Pandemic Legacy: Season 1',
  'Mutating diseases are spreading around the world - can your team save humanity?',
  '8.454',
  '8.61',
  '43380',
  '2021-08-11'),
 (3,
  '224517',
  'Brass: Birmingham',
  'Build networks, grow industries, and navigate the world of the Industrial Revolution.',
  '8.402',
  '8.67',
  '22412',
  '2021-08-11'),
 (4,
  '167791',
  'Terraforming Mars',
  'Compete with rival CEOs to make Mars habitable and build your corporate empire.',
  '8.279',
  '8.43',
  '69446',
  '2021-08-11'),
 (5,
  '291457',
  'Gloomhaven: Jaws of the Lion',
  'Vanquish monsters with strategic cardplay in a 25-scenario Gloomhaven campaign.',
  '8.256',
  '8.77',
  '12214',
  '2021-08-11'),
 (6,
  '233078',
  'Twilight Imperium: Fourth Edition',
  'Build an intergalactic empire through trade, rese

In [13]:
type(ranking_list)

list

In [14]:
ranking_list

[((1,
   '174430',
   'Gloomhaven',
   'Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!',
   '8.533',
   '8.77',
   '45001',
   '2021-08-11'),
  (2,
   '161936',
   'Pandemic Legacy: Season 1',
   'Mutating diseases are spreading around the world - can your team save humanity?',
   '8.454',
   '8.61',
   '43380',
   '2021-08-11'),
  (3,
   '224517',
   'Brass: Birmingham',
   'Build networks, grow industries, and navigate the world of the Industrial Revolution.',
   '8.402',
   '8.67',
   '22412',
   '2021-08-11'),
  (4,
   '167791',
   'Terraforming Mars',
   'Compete with rival CEOs to make Mars habitable and build your corporate empire.',
   '8.279',
   '8.43',
   '69446',
   '2021-08-11'),
  (5,
   '291457',
   'Gloomhaven: Jaws of the Lion',
   'Vanquish monsters with strategic cardplay in a 25-scenario Gloomhaven campaign.',
   '8.256',
   '8.77',
   '12214',
   '2021-08-11'),
  (6,
   '233078',
   'Twilight Imperium: Fourth Edition',
   'Build

In [15]:
game_ranking_1=pd.DataFrame(get_page(1),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_2=pd.DataFrame(get_page(2),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_3=pd.DataFrame(get_page(3),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_4=pd.DataFrame(get_page(4),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_5=pd.DataFrame(get_page(5),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_6=pd.DataFrame(get_page(6),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_7=pd.DataFrame(get_page(7),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_8=pd.DataFrame(get_page(8),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_9=pd.DataFrame(get_page(9),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])
game_ranking_10=pd.DataFrame(get_page(10),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])

In [16]:
game_ranking_1.head()

,rank,id,name,description,bggrating,avgrating,numvoter,date
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.533,8.77,45001,2021-08-11
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.454,8.61,43380,2021-08-11
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.402,8.67,22412,2021-08-11
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69446,2021-08-11
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12214,2021-08-11


In [17]:
game_ranking=pd.concat([game_ranking_1,
                        game_ranking_2,
                        game_ranking_3,
                        game_ranking_4,
                        game_ranking_5,
                        game_ranking_6,
                        game_ranking_7,
                        game_ranking_8,
                        game_ranking_9,
                        game_ranking_10], ignore_index=True)

In [18]:
game_ranking

,rank,id,name,description,bggrating,avgrating,numvoter,date
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.533,8.77,45001,2021-08-11
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.454,8.61,43380,2021-08-11
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.402,8.67,22412,2021-08-11
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69446,2021-08-11
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12214,2021-08-11
...,...,...,...,...,...,...,...,...
995,996,141423,Dead Men Tell No Tales,Work together as pirates to defeat enemies and...,6.527,7.13,3091,2021-08-11
996,997,244049,Forum Trajanum,Find out who's hiding in your Roman settlement...,6.527,7.32,2172,2021-08-11
997,998,204836,Escape Room: The Game,Solve puzzles and use the keys to stop the chr...,6.527,7.04,3420,2021-08-11
998,999,284818,Caylus 1303,Quarrel over action spaces (and player powers)...,6.526,7.60,1581,2021-08-11


In [20]:
game_ranking.to_csv('..\game_ranking.csv', index = True)
game_ranking

,rank,id,name,description,bggrating,avgrating,numvoter,date
0,1,174430,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,8.533,8.77,45001,2021-08-11
1,2,161936,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,8.454,8.61,43380,2021-08-11
2,3,224517,Brass: Birmingham,"Build networks, grow industries, and navigate ...",8.402,8.67,22412,2021-08-11
3,4,167791,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,8.279,8.43,69446,2021-08-11
4,5,291457,Gloomhaven: Jaws of the Lion,Vanquish monsters with strategic cardplay in a...,8.256,8.77,12214,2021-08-11
...,...,...,...,...,...,...,...,...
995,996,141423,Dead Men Tell No Tales,Work together as pirates to defeat enemies and...,6.527,7.13,3091,2021-08-11
996,997,244049,Forum Trajanum,Find out who's hiding in your Roman settlement...,6.527,7.32,2172,2021-08-11
997,998,204836,Escape Room: The Game,Solve puzzles and use the keys to stop the chr...,6.527,7.04,3420,2021-08-11
998,999,284818,Caylus 1303,Quarrel over action spaces (and player powers)...,6.526,7.60,1581,2021-08-11


In [44]:
gameid=game_ranking[['id']]
gameid

,id
0,174430
1,161936
2,224517
3,167791
4,291457
...,...
995,204836
996,141423
997,244049
998,284818


In [47]:
sampleid=game_ranking_1[['id']]
sampleid

,id
0,174430
1,161936
2,224517
3,167791
4,291457
...,...
95,203993
96,281259
97,178900
98,102680


In [50]:
sampleid=game_ranking_1['id'].to_list
sampleid

<bound method IndexOpsMixin.tolist of 0     174430
1     161936
2     224517
3     167791
4     291457
       ...  
95    203993
96    281259
97    178900
98    102680
99    132531
Name: id, Length: 100, dtype: object>